In [86]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate



In [87]:
load_dotenv()

True

In [92]:
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2b"
    #task="text-generation"
)

model = ChatHuggingFace(llm=llm)



In [93]:
result = model.invoke("create 2 days itenary to mumbai give 10 points only")

print(result.content)

StopIteration: 

In [ ]:
##Prompts - prompts are (super important) queries to guide models output 1.text based, 2. multi model prompt(image/sound/video)##

"""'
#What is temperature#
if temp= 0 it will give same output for each input
if  0.5 it will slightly change it for each input
as we increase it it will change for each input

"""

"'\n#What is temperature#\nif temp= 0 it will give same output for each input\nif we  set 0.5 it will slightly change it for each input\nas we increase it it will change for each input\n\n"

In [ ]:
#structured output

#Naukari.com -> resume -> wants to store(name, clg, batch, experience) 
#Here structured output will help to store retrived info in structured form
#When api building Product -> information -> (it will retrive imp info)
#Helps in agents (chatbots ill only talk but agents will do tasks) 

#we use with_structured_output before invoke
#1.TypedDict -> way to define a dict
# 2.Pydantic 
# 3.JsonSchema



In [9]:
from typing import  TypedDict

class Person(TypedDict):

    name: str
    age: int

new_person = Person = {'name' : 'rohan' , 'age' : '21'}

print(new_person)

{'name': 'rohan', 'age': '21'}


In [26]:
#reviews -> LLM -> generates Sentiment and Summary

from typing import TypedDict

from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate

load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct"
    #task="text-generation"
)

model = ChatHuggingFace(llm=llm)

model.invoke('who are you')

AIMessage(content='I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 13, 'total_tokens': 36}, 'model_name': 'meta-llama/Llama-3.1-8B-Instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--54377ce9-6d41-45ba-b2db-0821e13eaaf1-0', usage_metadata={'input_tokens': 13, 'output_tokens': 23, 'total_tokens': 36})

In [ ]:
#schema
class Review(TypedDict):

    summary: str
    sentiment: str

structured_model = model.with_structured_output(Review)

result = structured_model.invoke("""It is designed to be fast and efficient, offering significant speed for development and reducing human errors in the code.FastAPI automatically generates interactive API documentation using the OpenAPI standard, which makes it easy to understand and test your API without having to write extensive documentation manually.""")

print(type(result))

In [53]:

from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from typing import TypedDict, Annotated, Optional
from dotenv import load_dotenv

load_dotenv()

# Hugging Face LLM setup
llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-R1-0528")

model = ChatHuggingFace(llm=llm)

# Output schema
class Review(TypedDict):

    key_themes: Annotated[list[str], 'write all the key themes discussed in the review']
    summary: Annotated[str, 'A brief summary of the review']
    sentiment: Annotated[str, 'return sentiment of the review']
    pros: Annotated[Optional[list[str]], "write all the pros in Review"] 
    cons: Annotated[Optional[list[str]], "write all the cons in Review"] 


structured_model = model.with_structured_output(Review)

# Add an explicit instruction
input_text = """
It is designed to be fast and efficient, offering significant speed for development and reducing human errors in the code. FastAPI automatically generates interactive API documentation using the OpenAPI standard, which makes it easy to understand and test your API without having to write extensive documentation manually.
"""

result = structured_model.invoke(input_text)
print(result)


print(result["sentiment"])


{'key_themes': ['speed and efficiency', 'reduction of human errors', 'automated documentation'], 'summary': 'The tool highlights speed, efficiency, and automated documentation as key features of FastAPI.', 'sentiment': 'positive', 'pros': ['fast and efficient', 'reduces human errors', 'automatically generates interactive API documentation'], 'cons': []}
positive


In [ ]:
#Pydantic -> data parsing lib (data validation liberary)
#chat HuggingFace not supports it

from pydantic import BaseModel, Field
from typing import TypedDict, Literal


load_dotenv()

# Hugging Face LLM setup
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct")

model = ChatHuggingFace(llm=llm)

# Output schema
class Review(BaseModel):

    key_themes: list[str] = Field(description= 'write all the key themes discussed in the review')
    summary: str = Field(default= None, description= 'A brief summary of review')
    sentiment: Literal['pos', 'neg'] = Field( description= 'return sentiment of the review')
    pros: Optional[list[str]] = Field(default= None, description= 'write all the pros in Review')
    cons: Optional[list[str]] = Field(default= None, description= 'write all the cons in Review')
    name: Optional[str] = Field(default= None, description="write the name of the reviewer")


structured_model = model.with_structured_output(Review)

# Add an explicit instruction
input_text = """
It is designed to be fast and efficient, offering significant speed for development and reducing human errors in the code. FastAPI automatically generates interactive API documentation using the OpenAPI standard, which makes it easy to understand and test your API without having to write extensive documentation manually.
"""

result = structured_model.invoke(input_text)
print(result)


print(result.summary)
    



{}


Output Parsers

In [ ]:
#str op parsor
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

load_dotenv()
# Hugging Face LLM setup
llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-R1-0528")
model = ChatHuggingFace(llm=llm)


parser = StrOutputParser()

temp1 = PromptTemplate(
    template="write a detailed report on {topic}",
    input_variables=['topic']
)

temp2 = PromptTemplate(
    template="write a 5 line summary on following text. /n {text}",
    input_variables=["text"]
)

chain = temp1 | model | temp2 | model | parser

result = chain.invoke({'topic': 'back hole'})

print(result)

<think>
Okay, let me approach this systematically. The user has requested a 5-line summary of a detailed black hole report. First, I need to identify the absolute core elements from each section while respecting the line limit. 

Looking at the content structure, the most critical points appear to be: the definition (Intro), formation mechanism (Formation), key structural features (Structure), classification by mass (Types), and detection methods (Observation). 

The theoretical challenges section feels too specialized for this condensed format, but I should include practical detection evidence since it demonstrates real-world relevance. Properties like Hawking radiation might need to be implied rather than explicitly stated to save space. 

I'll start with a punchy definition that establishes what black holes fundamentally are. Then I'll focus on stellar death as the primary formation path - it's the most accessible scenario. For structure, the event horizon and singularity are non-ne

In [58]:
#json output parser(cant enforce schema)
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

template = PromptTemplate(
    template= "give me the name, age nd city of the fictional character \n {format_instruction}",
    input_variables=[],
    partial_variables={'format_instruction' : parser.get_format_instructions()}
)

chain  = template | model | parser

result = chain.invoke({})

print(result)

{'name': 'Evelyn Thorne', 'age': 34, 'city': 'Neo-Venezia'}


In [64]:
#structured output parser (cant do data validation)

from langchain.output_parsers import StructuredOutputParser, ResponseSchema

schema = [
    ResponseSchema(name='fact_1', description='fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='fact 3 about the topic'),
    ResponseSchema(name='fact_4', description='fact 4 about the topic')
]

parser = StructuredOutputParser.from_response_schemas(schema)

template = PromptTemplate(
    template='give 4 facts about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

result = chain.invoke({'topic': 'black hole'})

print(result)


{'fact_1': 'A black hole is an extremely dense region in space where gravity is so strong that nothing, not even light, can escape from it.', 'fact_2': "The event horizon is the boundary around a black hole beyond which no information or matter can escape, often called the 'point of no return'.", 'fact_3': 'Stellar black holes form when massive stars (typically more than 20 times the mass of the Sun) collapse under their own gravity at the end of their life cycle.', 'fact_4': 'Supermassive black holes, which are millions to billions of times the mass of our Sun, are found at the centers of most galaxies, including the Milky Way.'}


In [ ]:
#Pydantic output parser

#we want age alway greater than 18 and int





Chains
1. sequential chain(simple)
2. parallel chain
3. conditional chain

In [78]:
!pip install grandalf


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [83]:
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.output_parsers import StrOutputParser
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id='google/gemma-3n-E4B-it'
)

model = ChatHuggingFace(llm=llm)

prompt = PromptTemplate(
    template="generate 5 interesting facts about {topic}",
    input_variables=["topic"]
)

parser = StrOutputParser()

chain = prompt | model | parser

result = chain.invoke({'topic' : 'AI'})

print(result)

chain.get_graph().print_ascii()

StopIteration: 

In [84]:
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.output_parsers import StrOutputParser
load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id='deepseek-ai/DeepSeek-R1-0528'
)

model = ChatHuggingFace(llm=llm)

prompt = PromptTemplate(
    template="Generate detailed report on {topic}",
    input_variables=["topic"]
)

prompt2 = PromptTemplate(
    template="Generate 5 point summary from the following text \n {text}",
    input_variables=["text"]
)

parser = StrOutputParser()

chain1 = prompt | model | parser | prompt2 | model | parser

result = chain1.invoke({'topic' : 'AI'})

print(result)

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/fireworks-ai/inference/v1/chat/completions (Request ID: Root=1-6870f915-1b123df14363669b779d261c;111e2a8c-ed5b-46fd-b54b-82a4c096b66f)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

model = ChatOllama(model="gemma:2b")  

prompt1 = PromptTemplate(
    template="Generate detailed report on {topic}",
    input_variables=["topic"]
)

prompt2 = PromptTemplate(
    template="Generate 5 point summary from the following text \n {text}",
    input_variables=["text"]
)

parser = StrOutputParser()

chain1 = prompt1 | model | parser | prompt2 | model | parser

result = chain1.invoke({"topic": "AI"})
print(result)


C:\Users\Rohan\AppData\Local\Temp\ipykernel_22652\4035401300.py:9: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  model = ChatOllama(model="gemma:2b")  # or mistral/gemma/etc


Sure, here is a 5-point summary:

1. Artificial Intelligence (AI) is a broad field of study focused on creating machines that can perform tasks typically requiring human intelligence.


2. There are different types of AI, including narrow AI, general AI, reinforcement learning, and natural language processing.


3. AI has numerous applications across various industries, including healthcare, finance, transportation, manufacturing, customer service, and education.


4. AI faces challenges and ethical considerations such as bias and discrimination, job displacement, privacy, and transparency.


5. By addressing these issues, we can harness the power of AI to create a better future for all.


In [3]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

model = ChatOllama(model="gemma:2b")  

prompt1 = PromptTemplate(
    template="Generate detailed report on {topic}",
    input_variables=["topic"]
)

prompt2 = PromptTemplate(
    template="Generate 5 point summary from the following text \n {text}",
    input_variables=["text"]
)

parser = StrOutputParser()

chain1 = prompt1 | model | parser | prompt2 | model | parser

result = chain1.invoke({"topic": "AI"})
print(result)

chain1.get_graph().draw_ascii()


Sure, here's a 5-point summary of the text:

1. AI has a rich history, with origins in the 1950s.


2. AI exists in various forms, including narrow, general, and super AI.


3. AI possesses capabilities such as pattern recognition, natural language processing, image recognition, and data mining.


4. AI has numerous potential future applications in healthcare, finance, transportation, customer service, and manufacturing.


5. Challenges include bias, transparency, and job displacement due to AI automation.


'     +-------------+       \r\n     | PromptInput |       \r\n     +-------------+       \r\n            *              \r\n            *              \r\n            *              \r\n    +----------------+     \r\n    | PromptTemplate |     \r\n    +----------------+     \r\n            *              \r\n            *              \r\n            *              \r\n      +------------+       \r\n      | ChatOllama |       \r\n      +------------+       \r\n            *              \r\n            *              \r\n            *              \r\n   +-----------------+     \r\n   | StrOutputParser |     \r\n   +-----------------+     \r\n            *              \r\n            *              \r\n            *              \r\n+-----------------------+  \r\n| StrOutputParserOutput |  \r\n+-----------------------+  \r\n            *              \r\n            *              \r\n            *              \r\n    +----------------+     \r\n    | PromptTemplate |     \r\n    +--

In [17]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel
from dotenv import load_dotenv

load_dotenv()

model = ChatOllama(model="gemma:2b")  
model2 = ChatOllama(model='neural-chat:latest')

prompt1 = PromptTemplate(
    template="Generate short and simple notes for the following text \n {text}",
    input_variables=["text"]
)

prompt2 = PromptTemplate(
    template="Generate 5 short questions from the following text \n {text}",
    input_variables=["text"]
)

prompt3 = PromptTemplate(
    template='merge the provided notes and quize into a single document \n notes -> {notes} and quize -> {quize} ',
    input_variables=['notes', 'quize']
)

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    'notes' : prompt1 | model | parser,
    'quize' : prompt2 | model2 | parser
})

merge_chain = prompt3 | model | parser

chain = parallel_chain | merge_chain

text = """After working as an office assistant for T-Series as well as assistant director in Vikram Bhatt's films Kasoor (2001), Awara Paagal Deewana (2002) and Footpath (2003), Suri made his directorial debut with the moderately successful Zeher (2005) and then directed movies like Kalyug (2005), Woh Lamhe (2006), Awarapan (2007), Raaz: The Mystery Continues (2009) and Crook (2010).[6]
Suri's breakthrough period begun with the unexpected earning of his psychological thriller Murder 2 (2011). It is one of the highest grossing Hindi film of 2011. Followed by the highly successful musical love stories Aashiqui 2 (2013) and Ek Villain (2014), with the latter being a revenge drama too and entering 100 Crore Club in India.[7] Post the debacle of his majorly anticipated dramas Hamari Adhuri Kahani (2015) and Half Girlfriend (2017), he garnered critical and commercial success via the romantic suspense thriller Malang (2020).[8] Recently released Ek Villain Returns (2022) with John Abraham and Arjun Kapoor.
Suri's upcoming and forthcoming directorials include, Untitled movie with Varun Dhawan and Malang 2.[9] His upcoming movie is "Saiyaara" with debutant Ahaan Panday and Aneet Padda under Yash Raj Films set to release on July 18, 2025."""

result = chain.invoke({"text": text})
print(result)

chain.get_graph().draw_ascii()


## Suri's Directorial Journey

**Debut and First Film:**
* Made his directorial debut in 2005 with **Zeher**.

**Critical and Commercial Success:**
* Gained critical and commercial success with **Mangal (2020)**.

**Upcoming and Forthcoming Projects:**
* "Untitled movie with Varun Dhawan and Malang 2".
* "Saiyaara" with debutant Ahaan Panday and Aneet Padda under Yash Raj Films set to release on July 18, 2025.

**Assistant Directorial Work:**
* Details of his assistant directorial work are not provided in the context.

**Hamari Adhuri Kahani and Half Girlfriend:**
* The context does not mention any information about Hamari Adhuri Kahani or Half Girlfriend.

**Ek Villain Returns:**
* The context does not provide any information about the main stars or upcoming movies planned by Suri for that film.


'           +----------------------------+            \r\n           | Parallel<notes,quize>Input |            \r\n           +----------------------------+            \r\n                 **               **                 \r\n              ***                   ***              \r\n            **                         **            \r\n+----------------+                +----------------+ \r\n| PromptTemplate |                | PromptTemplate | \r\n+----------------+                +----------------+ \r\n          *                               *          \r\n          *                               *          \r\n          *                               *          \r\n  +------------+                    +------------+   \r\n  | ChatOllama |                    | ChatOllama |   \r\n  +------------+                    +------------+   \r\n          *                               *          \r\n          *                               *          \r\n          *                   

In [16]:
#Conditional Chains

from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv

load_dotenv()

model = ChatOllama(model="gemma:2b")  

parser = StrOutputParser()

class Feedback(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

parser2 = PydanticOutputParser(pydantic_object=Feedback)

prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

classifier_chain = prompt1 | model | parser2

prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

chain = classifier_chain | branch_chain

print(chain.invoke({'feedback': 'This is a beautiful phone'}))

chain.get_graph().print_ascii()

OutputParserException: Failed to parse Feedback from completion {"properties": {"sentiment": {"description": "Give the sentiment of the feedback", "enum": ["positive", "negative"], "title": "Sentiment", "type": "string"}}}. Got: 1 validation error for Feedback
sentiment
  Field required [type=missing, input_value={'properties': {'sentimen...nt', 'type': 'string'}}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

Runnables

In [ ]:
class Naklillm():
    def __init__(self):
        print("llm created")

    def predict(self, promptt):

        response_list = [
            
        ]